In [2]:
import gym 
from gym import spaces
import numpy as np
import pygame
from pygame.locals import *
from run import GameController
from constants import *
from pacman import Pacman
from ghost import Ghosts
from nodes import NodeGroup
from pellets import PelletGroup
from fruits import Fruits
from stable_baselines3 import DQN
from stable_baselines3 import PPO
import os

class PacmanEnv(gym.Env):
    metadata = {"render.modes": ["human"]}

class PacmanEnv(gym.Env):
    def __init__(self, render_mode=False):
        super(PacmanEnv, self).__init__()
        self.render_mode = render_mode

        if not render_mode:
            os.environ["SDL_VIDEODRIVER"] = "dummy"
        else:
            os.environ.pop("SDL_VIDEODRIVER", None)

        pygame.quit() 
        pygame.init()

        self.game = GameController(render_mode=render_mode)

        self.action_space = spaces.Discrete(5, start=-2)
        self.observation_space = spaces.Box(
            low=0, high=255, shape=(SCREENHEIGHT, SCREENWIDTH, 3), dtype=np.uint8
        )

    def reset(self):
        self.game.startGame(3)
        state = self.get_observation()
        return state
    
    def step(self, action):
        action = int(action)
        action = np.clip(action, 0, 4)
    
        action = action - 2
        if self.game.pacman.validDirection(action):
            self.game.pacman.direction = action 

        pelletBefore = self.game.pellets.numEaten 
        lifesBefore = self.game.pacman.life_amount   
        self.game.update()

        if self.game.pacman.target is not None and self.game.pacman.overshotTarget():
            self.game.pacman.node = self.game.pacman.target
            self.game.pacman.setPosition()

        self.game.update() 

        state = self.get_observation() 

        reward = 0

        pellet = self.game.pellets.numEaten - pelletBefore
        if pellet == 1:
            reward += 20

        fruit = None
        if self.game.fruits is not None:
            fruit = self.game.pacman.eatFruits(self.game.fruits)
            if fruit:
                reward += 20

        lifes = self.game.pacman.life_amount - lifesBefore
        if lifes == -1:
            reward -= 50


        if pellet == 0 and fruit is None:
            reward -= 2

        done = self.check_game_over()

        info = {}

        return state, reward, done, info
    
    def render(self, mode="human"):
        if self.render_mode and mode == "human":
            self.game.render()

    def get_observation(self):
        return pygame.surfarray.array3d(self.game.screen)

    def _init_pygame(self):
        if not pygame.get_init():
            pygame.init()

    def close(self):
        pygame.quit()

    def check_game_over(self):
        return self.game.pacman.life_amount == 0
    
    def change_resolution(self, width, height):
        global SCREENWIDTH, SCREENHEIGHT

        constants_path = os.path.join(os.path.dirname(__file__), "constants.py")
        with open(constants_path, "r") as file:
            lines = file.readlines()

        with open(constants_path, "w") as file:
            for line in lines:
                if line.startswith("SCREENWIDTH"):
                    file.write(f"SCREENWIDTH = {width}\n")
                elif line.startswith("SCREENHEIGHT"):
                    file.write(f"SCREENHEIGHT = {height}\n")
                else:
                    file.write(line)

        SCREENWIDTH, SCREENHEIGHT = width, height
        
        self.game.screen = pygame.display.set_mode((SCREENWIDTH, SCREENHEIGHT))
        self.game.width, self.game.height = SCREENWIDTH, SCREENHEIGHT

    def get_observation(self):
        observation = pygame.surfarray.array3d(self.game.screen)
        return np.transpose(observation, (1, 0, 2))








In [7]:
env = PacmanEnv(render_mode=False)
model = DQN("MlpPolicy", env, verbose=1, buffer_size=1000)

print("Training the model...")
model.learn(total_timesteps=2000)
model.save("pacman_dqn_model")
env.close()



Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Training the model...
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 193      |
|    ep_rew_mean      | 600      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1        |
|    time_elapsed     | 662      |
|    total_timesteps  | 772      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 6.28     |
|    n_updates        | 167      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 161      |
|    ep_rew_mean      | 834      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1        |
|    time_elapsed     | 1168     |
|  

In [ ]:
print("Switching to testing mode...")
env = PacmanEnv(render_mode=True)
state = env.reset()
rewardMain = 0
model = DQN("MlpPolicy", env, verbose=1, buffer_size=1000)

model.load("pacman_dqn_model.zip")

state = env.reset()

rewards = []
durations = []
episodes = 15

for episode in range(episodes):
    state = env.reset()
    episode_reward = 0
    steps = 0
    done = False

    while not done:
        action, _ = model.predict(state)
        state, reward, done, _ = env.step(action)
        episode_reward += reward
        steps += 1

    rewards.append(episode_reward)
    durations.append(steps)

file = open("tests.txt")
file.write("DQN model for {episodes} episodes")    
for i in range(rewards.count()):
    file.write("episode {i}:", "reward ",rewards[i], " duration: ", durations[i])    
    
env.close()



Switching to testing mode...
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
-1978


In [1]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

env = PacmanEnv(render_mode=False)
    
model = PPO("CnnPolicy", env, verbose=1, n_steps=256, batch_size=64, ent_coef=0.01)

print("Training the PPO model...")
model.learn(total_timesteps=100)
model.save("pacman_ppo_model")
env.close() 

NameError: name 'PacmanEnv' is not defined

In [ ]:
print("Switching to testing mode...")
env = PacmanEnv(render_mode=True)  # New environment with rendering enabled
state = env.reset()
rewardMain = 0
model = PPO("CnnPolicy", env, verbose=1, n_steps=256, batch_size=64, ent_coef=0.01)

model.load("pacman_ppo_model.zip")

state = env.reset()

rewards = []
durations = []
episodes = 15

for episode in range(episodes):
    state = env.reset()
    episode_reward = 0
    steps = 0
    done = False

    while not done:
        action, _ = model.predict(state)
        state, reward, done, _ = env.step(action)
        episode_reward += reward
        steps += 1

    rewards.append(episode_reward)
    durations.append(steps)

file = open("tests.txt")
file.write("ppo model for {episodes} episodes")    
for i in range(rewards.count()):
    file.write("episode {i}:", "reward ",rewards[i], " duration: ", durations[i])    
    
env.close()